In [2]:
# [1] 준비: PyTorch 설치
!pip install torch --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.5 MB/s eta 0:00:00


In [4]:

# [2] 한글 데이터셋 예시 (직접 가사, 명언 등 추가 가능)
corpus = [
    "오늘도 좋은 하루 되세요",
    "행복은 마음속에 있어요",
    "내일은 더 나아질 거예요",
    "희망은 포기하지 않는 마음입니다",
    "슬픔은 곧 지나갈 거예요",
    "사랑은 모든 것을 이겨냅니다",
    "지금 이 순간을 소중히 하세요",
    "용기는 두려움을 이기는 힘입니다",
    "고마운 마음을 잊지 마세요",
    "친구는 소중한 보물입니다"
]
text = ' '.join(corpus)
words = list(set(text.split()))
word2idx = {w: i for i, w in enumerate(words)}
idx2word = {i: w for w, i in word2idx.items()}

# [3] 시퀀스 데이터 생성 (2-gram 기준)
seq_length = 2
sequences = []
for sent in corpus:
    tokens = sent.split()
    for i in range(len(tokens) - seq_length):
        seq = tokens[i:i+seq_length+1]
        print(seq)
        sequences.append([word2idx[w] for w in seq])

import torch
X = [s[:-1] for s in sequences]
y = [s[-1] for s in sequences]
X = torch.tensor(X)
y = torch.tensor(y)

print(X.shape)

# [4] LSTM 언어모델 정의
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.embed(x)
        _, (h, _) = self.lstm(x)
        out = self.fc(h.squeeze(0))
        return out

# [5] 모델 학습
model = LSTMModel(len(words), 16, 32)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(300):
    optimizer.zero_grad()
    out = model(X)
    loss = loss_fn(out, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}: loss={loss.item():.4f}")

# [6] 문장 생성 함수
def generate(seed_text, max_len=5):
    tokens = seed_text.split()
    for _ in range(max_len):
        seq = [word2idx.get(w, 0) for w in tokens[-2:]]
        inp = torch.tensor([seq])
        out = model(inp)
        pred = out.argmax().item()
        next_word = idx2word[pred]
        tokens.append(next_word)
    return ' '.join(tokens)

# [7] 문장 생성 예시
print("생성 예시:", generate("오늘도 좋은"))
print("생성 예시:", generate("희망은 포기하지"))
print("생성 예시:", generate("고마운 마음을"))


['오늘도', '좋은', '하루']
['좋은', '하루', '되세요']
['행복은', '마음속에', '있어요']
['내일은', '더', '나아질']
['더', '나아질', '거예요']
['희망은', '포기하지', '않는']
['포기하지', '않는', '마음입니다']
['슬픔은', '곧', '지나갈']
['곧', '지나갈', '거예요']
['사랑은', '모든', '것을']
['모든', '것을', '이겨냅니다']
['지금', '이', '순간을']
['이', '순간을', '소중히']
['순간을', '소중히', '하세요']
['용기는', '두려움을', '이기는']
['두려움을', '이기는', '힘입니다']
['고마운', '마음을', '잊지']
['마음을', '잊지', '마세요']
['친구는', '소중한', '보물입니다']
torch.Size([19, 2])
Epoch 50: loss=0.0125
Epoch 100: loss=0.0036
Epoch 150: loss=0.0022
Epoch 200: loss=0.0015
Epoch 250: loss=0.0011
Epoch 300: loss=0.0009
생성 예시: 오늘도 좋은 하루 되세요 힘입니다 보물입니다 거예요
생성 예시: 희망은 포기하지 않는 마음입니다 마음입니다 되세요 힘입니다
생성 예시: 고마운 마음을 잊지 마세요 마세요 않는 마음입니다
